# Descripción del proyecto  


La compañía de seguros Sure Tomorrow quiere resolver varias tareas con la ayuda de machine learning y te pide que evalúes esa posibilidad.

Tarea 1: encontrar clientes que sean similares a un cliente determinado. Esto ayudará a los agentes de la compañía con el marketing.  
Tarea 2: predecir si es probable que un nuevo cliente reciba un beneficio de seguro. ¿Puede un modelo de predicción entrenado funcionar mejor que un modelo dummy no entrenado? ¿Puede funcionar peor? Explica tu respuesta.  
Tarea 3: predecir la cantidad de beneficios de seguro que probablemente recibirá un nuevo cliente utilizando un modelo de regresión lineal.  
Tarea 4: proteger los datos personales de los clientes sin romper el modelo de la tarea anterior.  

Es necesario desarrollar un algoritmo de transformación de datos que dificulte la recuperación de la información personal si los datos caen en manos equivocadas. Esto se denomina enmascaramiento de datos u ofuscación de datos. Pero los datos deben protegerse de tal manera que la calidad de los modelos de machine learning no se vea afectada. No es necesario elegir el mejor modelo, basta con demostrar que el algoritmo funciona correctamente.

### Instrucciones del proyecto  

1. arga los datos.  
2. Verifica que los datos no tengan problemas: no faltan datos, no hay valores extremos, etc.  
3. Trabaja en cada tarea y responde las preguntas planteadas en la plantilla del proyecto.  
4. Saca conclusiones basadas en tu experiencia trabajando en el proyecto.  

Hay algo de código previo en la plantilla del proyecto, siéntete libre de usarlo. Primero se debe terminar algo de código previo. Además, hay dos apéndices en la plantilla del proyecto con información útil.  

### Descripción de datos  

El dataset se almacena en el archivo /datasets/insurance_us.csv.  

Características: sexo, edad, salario y número de familiares de la persona asegurada.  
Objetivo: número de beneficios de seguro recibidos por una persona asegurada en los últimos cinco años.  

### Evaluación del proyecto  

Hemos definido los criterios de evaluación para el proyecto. Léelos con atención antes de pasar al ejercicio.

Esto es en lo que se fijarán los revisores al examinar tu proyecto:  

¿Seguiste todos los pasos de las instrucciones?  
¿Mantuviste la estructura del proyecto?  
¿Mantuviste el código ordenado?  
¿Desarrollaste todos los procedimientos necesarios y respondiste todas las preguntas?  
¿Sacaste tus conclusiones?  

# Preprocesamiento y exploración de datos  

### 1  Inicialización  

In [ ]:
pip install scikit-learn --upgrade

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns

import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.preprocessing

from sklearn.model_selection import train_test_split

from IPython.display import display

# 2  Carga de datos  

Carga los datos y haz una revisión básica para comprobar que no hay problemas obvios.

In [ ]:
df = pd.read_csv('/datasets/insurance_us.csv')

Renombramos las columnas para que el código se vea más coherente con su estilo.

In [ ]:
df = df.rename(columns={'Gender': 'gender', 'Age': 'age', 'Salary': 'income', 'Family members': 'family_members', 'Insurance benefits': 'insurance_benefits'})

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
# puede que queramos cambiar el tipo de edad (de float a int) aunque esto no es crucial

# escribe tu conversión aquí si lo deseas:



In [ ]:
# comprueba que la conversión se haya realizado con éxito

In [ ]:
# ahora echa un vistazo a las estadísticas descriptivas de los datos.# ¿Se ve todo bien?

# 3  Análisis exploratorio de datos

Vamos a comprobar rápidamente si existen determinados grupos de clientes observando el gráfico de pares.

In [ ]:
g = sns.pairplot(df, kind='hist')
g.fig.set_size_inches(12, 12)

De acuerdo, es un poco complicado detectar grupos obvios (clústeres) ya que es difícil combinar diversas variables simultáneamente (para analizar distribuciones multivariadas). Ahí es donde LA y ML pueden ser bastante útiles.

# Tarea 1. Clientes similares

En el lenguaje de ML, es necesario desarrollar un procedimiento que devuelva los k vecinos más cercanos (objetos) para un objeto dado basándose en la distancia entre los objetos. Es posible que quieras revisar las siguientes lecciones (capítulo -> lección)- Distancia entre vectores -> Distancia euclidiana

* Distancia entre vectores -> Distancia Manhattan  

Para resolver la tarea, podemos probar diferentes métricas de distancia.

Escribe una función que devuelva los k vecinos más cercanos para un  𝑛𝑡ℎ  
  objeto basándose en una métrica de distancia especificada. A la hora de realizar esta tarea no debe tenerse en cuenta el número de prestaciones de   seguro recibidas. Puedes utilizar una implementación ya existente del algoritmo kNN de scikit-learn (consulta el enlace) o tu propia implementación.   Pruébalo para cuatro combinaciones de dos casos- Escalado  

* los datos no están escalados  
* los datos se escalan con el escalador MaxAbsScaler  
* Métricas de distancia  
* Euclidiana  
* Manhattan  

Responde a estas preguntas:- ¿El hecho de que los datos no estén escalados afecta al algoritmo kNN? Si es así, ¿cómo se manifiesta?- ¿Qué tan similares son los resultados al utilizar la métrica de distancia Manhattan (independientemente del escalado)?  

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

In [ ]:
def get_knn(df, n, k, metric):
    
    """
    Devuelve los k vecinos más cercanos

    :param df: DataFrame de pandas utilizado para encontrar objetos similares dentro del mismo lugar    :param n: número de objetos para los que se buscan los vecinos más cercanos    :param k: número de vecinos más cercanos a devolver
    :param métrica: nombre de la métrica de distancia    """

    nbrs = # <tu código aquí> 
    nbrs_distances, nbrs_indices = nbrs.kneighbors([df.iloc[n][feature_names]], k, return_distance=True)
    
    df_res = pd.concat([
        df.iloc[nbrs_indices[0]], 
        pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])
        ], axis=1)
    
    return df_res

Escalar datos.

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

transformer_mas = sklearn.preprocessing.MaxAbsScaler().fit(df[feature_names].to_numpy())

df_scaled = df.copy()
df_scaled.loc[:, feature_names] = transformer_mas.transform(df[feature_names].to_numpy())

In [ ]:
df_scaled.sample(5)

Ahora, vamos a obtener registros similares para uno determinado, para cada combinación

Respuestas a las preguntas

¿El hecho de que los datos no estén escalados afecta al algoritmo kNN? Si es así, ¿cómo se manifiesta?

Escribe tu respuesta aquí.

¿Qué tan similares son los resultados al utilizar la métrica de distancia Manhattan (independientemente del escalado)?

Escribe tu respuesta aquí.

# Tarea 2. ¿Es probable que el cliente reciba una prestación del seguro?

En términos de machine learning podemos considerarlo como una tarea de clasificación binaria.


Con el valor de insurance_benefits superior a cero como objetivo, evalúa si el enfoque de clasificación kNN puede funcionar mejor que el modelo dummy. Instrucciones:  

* Construye un clasificador basado en KNN y mide su calidad con la métrica F1 para k=1...10 tanto para los datos originales como para los escalados. Sería interesante observar cómo k puede influir en la métrica de evaluación y si el escalado de los datos provoca alguna diferencia. Puedes utilizar una implementación ya existente del algoritmo de clasificación kNN de scikit-learn (consulta el enlace) o tu propia implementación.- Construye un modelo dummy que, en este caso, es simplemente un modelo aleatorio. Debería devolver "1" con cierta probabilidad. Probemos el modelo con cuatro valores de probabilidad: 0, la probabilidad de pagar cualquier prestación del seguro, 0.5, 1. La probabilidad de pagar cualquier prestación del seguro puede definirse como  

𝑃{prestación de seguro recibida}=número de clientes que han recibido alguna prestación de seguro / número total de clientes.  
 
Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30.  

In [ ]:
# сalcula el objetivo
df['insurance_benefits_received'] = #<tu código aquí>

In [ ]:
# comprueba el desequilibrio de clases con value_counts()

# <tu código aquí>

In [ ]:
def eval_classifier(y_true, y_pred):
    
    f1_score = sklearn.metrics.f1_score(y_true, y_pred)
    print(f'F1: {f1_score:.2f}')
    
# si tienes algún problema con la siguiente línea, reinicia el kernel y ejecuta el cuaderno de nuevo    cm = sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='all')
    print('Matriz de confusión')
    print(cm)

In [ ]:
# generar la salida de un modelo aleatorio

def rnd_model_predict(P, size, seed=42):

    rng = np.random.default_rng(seed=seed)
    return rng.binomial(n=1, p=P, size=size)

In [ ]:
for P in [0, df['insurance_benefits_received'].sum() / len(df), 0.5, 1]:

    print(f'La probabilidad: {P:.2f}')
    y_pred_rnd = # <tu código aquí> 
        
    eval_classifier(df['insurance_benefits_received'], y_pred_rnd)
    
    print()

# Tarea 3. Regresión (con regresión lineal)

Con insurance_benefits como objetivo, evalúa cuál sería la RECM de un modelo de regresión lineal.

Construye tu propia implementación de regresión lineal. Para ello, recuerda cómo está formulada la solución de la tarea de regresión lineal en términos de LA. Comprueba la RECM tanto para los datos originales como para los escalados. ¿Puedes ver alguna diferencia en la RECM con respecto a estos dos casos?  

Denotemos-  𝑋: matriz de características; cada fila es un caso, cada columna es una característica, la primera columna está formada por unidades-  𝑦 — objetivo (un vector)-  𝑦̂ — objetivo estimado (un vector)-  𝑤 — vector de pesos La tarea de regresión lineal en el lenguaje de las matrices puede formularse así:  

𝑦=𝑋𝑤  

El objetivo de entrenamiento es entonces encontrar esa  𝑤  w que minimice la distancia L2 (ECM) entre  𝑋𝑤  y 𝑦:  

min𝑤𝑑2(𝑋𝑤,𝑦) or min𝑤MSE(𝑋𝑤,𝑦)  
 

Parece que hay una solución analítica para lo anteriormente expuesto:  

𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦  
 

La fórmula anterior puede servir para encontrar los pesos  𝑤 y estos últimos pueden utilizarse para calcular los valores predichos  

𝑦̂ =𝑋𝑣𝑎𝑙𝑤  

Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30. Utiliza la métrica RECM para evaluar el modelo.

In [ ]:
class MyLinearRegression:
    
    def __init__(self):
        
        self.weights = None
    
    def fit(self, X, y):
        
        # añadir las unidades
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        self.weights = # <tu código aquí>

    def predict(self, X):
        
        # añadir las unidades
        X2 = # <tu código aquí>
        y_pred = # <tu código aquí>
        
        return y_pred

In [ ]:
def eval_regressor(y_true, y_pred):
    
    rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    print(f'RMSE: {rmse:.2f}')
    
    r2_score = math.sqrt(sklearn.metrics.r2_score(y_true, y_pred))
    print(f'R2: {r2_score:.2f}')    

In [ ]:
X = df[['age', 'gender', 'income', 'family_members']].to_numpy()
y = df['insurance_benefits'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

lr = MyLinearRegression()

lr.fit(X_train, y_train)
print(lr.weights)

y_test_pred = lr.predict(X_test)
eval_regressor(y_test, y_test_pred)

# Tarea 4. Ofuscar datos

Lo mejor es ofuscar los datos multiplicando las características numéricas (recuerda que se pueden ver como la matriz  𝑋) por una matriz invertible  𝑃.  

𝑋′=𝑋×𝑃  
 

Trata de hacerlo y comprueba cómo quedarán los valores de las características después de la transformación. Por cierto, la propiedad de invertibilidad es importante aquí, así que asegúrate de que  𝑃 sea realmente invertible.  

Puedes revisar la lección 'Matrices y operaciones matriciales -> Multiplicación de matrices' para recordar la regla de multiplicación de matrices y su implementación con NumPy.  

In [ ]:
personal_info_column_list = ['gender', 'age', 'income', 'family_members']
df_pn = df[personal_info_column_list]

In [ ]:
X = df_pn.to_numpy()

Generar una matriz aleatoria  𝑃.  

In [ ]:
rng = np.random.default_rng(seed=42)
P = rng.random(size=(X.shape[1], X.shape[1]))

Comprobar que la matriz P sea invertible

¿Puedes adivinar la edad o los ingresos de los clientes después de la transformación?

¿Puedes recuperar los datos originales de  𝑋′ si conoces  𝑃 ? Intenta comprobarlo a través de los cálculos moviendo  𝑃 del lado derecho de la fórmula anterior al izquierdo. En este caso las reglas de la multiplicación matricial son realmente útiles.

Muestra los tres casos para algunos clientes- Datos originales  
  
El que está transformado- El que está invertido (recuperado)  

Seguramente puedes ver que algunos valores no son exactamente iguales a los de los datos originales. ¿Cuál podría ser la razón de ello?

# 4  Prueba de que la ofuscación de datos puede funcionar con regresión lineal

En este proyecto la tarea de regresión se ha resuelto con la regresión lineal. Tu siguiente tarea es demostrar analytically que el método de ofuscación no afectará a la regresión lineal en términos de valores predichos, es decir, que sus valores seguirán siendo los mismos. ¿Lo puedes creer? Pues no hace falta que lo creas, ¡tienes que que demostrarlo!

Entonces, los datos están ofuscados y ahora tenemos  𝑋×𝑃 en lugar de tener solo  𝑋. En consecuencia, hay otros pesos  𝑤𝑃 como  

𝑤=(𝑋𝑇𝑋)−1𝑋𝑇𝑦⇒𝑤𝑃=[(𝑋𝑃)𝑇𝑋𝑃]−1(𝑋𝑃)𝑇𝑦  
 

¿Cómo se relacionarían  𝑤 y 𝑤𝑃 si simplificáramos la fórmula de  𝑤𝑃 anterior?  

¿Cuáles serían los valores predichos con  𝑤𝑃?  

¿Qué significa esto para la calidad de la regresión lineal si esta se mide mediante la RECM? Revisa el Apéndice B Propiedades de las matrices al final del cuaderno. ¡Allí encontrarás fórmulas muy útiles!  

No es necesario escribir código en esta sección, basta con una explicación analítica.  

Respuesta

Type Markdown and LaTeX:  𝛼2

Prueba analítica

Type Markdown and LaTeX:  𝛼2

# 5  Prueba de regresión lineal con ofuscación de datos

Ahora, probemos que la regresión lineal pueda funcionar, en términos computacionales, con la transformación de ofuscación elegida. Construye un procedimiento o una clase que ejecute la regresión lineal opcionalmente con la ofuscación. Puedes usar una implementación de regresión lineal de scikit-learn o tu propia implementación. Ejecuta la regresión lineal para los datos originales y los ofuscados, compara los valores predichos y los valores de las métricas RMSE y  𝑅2  
 . ¿Hay alguna diferencia?  

Procedimiento  
  
* Crea una matriz cuadrada  𝑃 de números aleatorios.- Comprueba que sea invertible. Si no lo es, repite el primer paso hasta obtener una matriz   
invertible.- <¡ tu comentario aquí !>  
* Utiliza  𝑋𝑃 como la nueva matriz de características  

# Conclusiones

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter.

 * Jupyter Notebook está abierto  
 * El código no tiene errores- [ ] Las celdas están ordenadas de acuerdo con la lógica y el orden de ejecución  
 * Se ha realizado la tarea 1  
     * Está presente el procedimiento que puede devolver k clientes similares para un cliente determinado  
     * Se probó el procedimiento para las cuatro combinaciones propuestas - [ ] Se respondieron las preguntas sobre la escala/distancia- [ ] Se ha realizado la tarea 2  
     * Se construyó y probó el modelo de clasificación aleatoria para todos los niveles de probabilidad - [ ] Se construyó y probó el modelo de clasificación kNN tanto para los datos originales como para los escalados. Se calculó la métrica F1.- [ ] Se ha realizado la tarea 3  
     * Se implementó la solución de regresión lineal mediante operaciones matriciales - [ ] Se calculó la RECM para la solución implementada- [ ] Se ha realizado la tarea 4  
     * Se ofuscaron los datos mediante una matriz aleatoria e invertible P - [ ] Se recuperaron los datos ofuscados y se han mostrado algunos ejemplos - [ ] Se proporcionó la prueba analítica de que la transformación no afecta a la RECM - [ ] Se proporcionó la prueba computacional de que la transformación no afecta a la RECM- [ ] Se han sacado conclusiones

# Apéndices
## 6  Apéndice A: Escribir fórmulas en los cuadernos de Jupyter

Puedes escribir fórmulas en tu Jupyter Notebook utilizando un lenguaje de marcado proporcionado por un sistema de publicación de alta calidad llamado  𝐿𝐴𝑇𝐸𝑋 (se pronuncia como "Lah-tech"). Las fórmulas se verán como las de los libros de texto.  

Para incorporar una fórmula a un texto, pon el signo de dólar ($) antes y después del texto de la fórmula, por ejemplo:  12×32=34 or 𝑦=𝑥2,𝑥≥1.  
  
Si una fórmula debe estar en el mismo párrafo, pon el doble signo de dólar ($$) antes y después del texto de la fórmula, por ejemplo:  

𝑥¯=1𝑛∑𝑖=1𝑛𝑥𝑖.  


El lenguaje de marcado de LaTeX es muy popular entre las personas que utilizan fórmulas en sus artículos, libros y textos. Puede resultar complicado, pero sus fundamentos son sencillos. Consulta esta ficha de ayuda (materiales en inglés) de dos páginas para aprender a componer las fórmulas más comunes.

# 7  Apéndice B: Propiedades de las matrices

Las matrices tienen muchas propiedades en cuanto al álgebra lineal. Aquí se enumeran algunas de ellas que pueden ayudarte a la hora de realizar la prueba analítica de este proyecto.

Distributividad	 𝐴(𝐵+𝐶)=𝐴𝐵+𝐴𝐶  
 
No conmutatividad	 𝐴𝐵≠𝐵𝐴  
 
Propiedad asociativa de la multiplicación	 (𝐴𝐵)𝐶=𝐴(𝐵𝐶)  
 
Propiedad de identidad multiplicativa	 𝐼𝐴=𝐴𝐼=𝐴  
 
𝐴−1𝐴=𝐴𝐴−1=𝐼  
 
(𝐴𝐵)−1=𝐵−1𝐴−1  
 
Reversibilidad de la transposición de un producto de matrices,	 (𝐴𝐵)𝑇=𝐵𝑇𝐴𝑇  